# 1. Cleaning_for_one_file

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('demo_uncleaned.csv')

# Step 1: Handle Missing Values
# Reconfirm if there are any missing values in the original dataset and add those rows to rows_to_remove
data_cleaned = data.copy()
rows_to_remove = set(data_cleaned[data_cleaned.isna().any(axis=1)].index)
missing_values_indices = list(rows_to_remove)
missing_values_count = len(missing_values_indices)

# Step 2: Remove Rows with Negative Values
# Assuming that negative values are not valid for this dataset, add rows with negative values to rows_to_remove
for col in data_cleaned.select_dtypes(include='number').columns:
    negative_indices = data_cleaned[data_cleaned[col] < 0].index
    rows_to_remove.update(negative_indices)
negative_values_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices]
negative_values_count = len(negative_values_indices)

# Step 3: Handle Outliers by Removing Rows with Outliers (Based on Row Statistics)
# Define outliers as values beyond 3 standard deviations from the mean for each row
numeric_cols = data_cleaned.select_dtypes(include='number').columns
for idx, row in data_cleaned[numeric_cols].iterrows():
    row_mean = row.mean()
    row_std_dev = row.std()
    outlier_threshold = 4 * row_std_dev

    # Check if any value in the row is an outlier
    if any((row > row_mean + outlier_threshold) | (row < row_mean - outlier_threshold)):
        rows_to_remove.add(idx)

outliers_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices]
outliers_count = len(outliers_indices)

# Step 4: Remove Rows with All Zeros (for numeric columns) and add them to rows_to_remove
zero_rows = data_cleaned[numeric_cols].eq(0).all(axis=1)
rows_to_remove.update(data_cleaned[zero_rows].index)
all_zeros_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices and idx not in outliers_indices]
all_zeros_count = len(all_zeros_indices)

# Step 5: Remove Duplicate Records
# Find and add duplicate rows to rows_to_remove
duplicates = data_cleaned[data_cleaned.duplicated(keep=False)].index
rows_to_remove.update(duplicates)
duplicates_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices and idx not in outliers_indices and idx not in all_zeros_indices]
duplicates_count = len(duplicates_indices)

# Convert rows_to_remove to a list and sort it
rows_to_remove = sorted(list(rows_to_remove))

# Remove all identified rows from data_cleaned
data_cleaned = data_cleaned.drop(index=rows_to_remove, errors='ignore')

# Extract the removed rows from the original dataset
removed_rows = data.loc[rows_to_remove]

# Convert the removed rows to a numpy matrix
removed_rows_matrix = removed_rows.to_numpy()

# Save the cleaned data and removed rows into new Excel files
cleaned_file_path = 'cleaned_reshaped_merged_8789.xlsx'
removed_rows_file_path = 'removed_rows_reshaped_merged_8789.xlsx'

# Save the cleaned data to an Excel file
data_cleaned.to_excel(cleaned_file_path, index=False)

# Save the removed rows to a separate Excel file
removed_rows.to_excel(removed_rows_file_path, index=False)

# Output the file paths and counts for user's reference
print(f"Cleaned data saved to: {cleaned_file_path}")
print(f"Removed rows saved to: {removed_rows_file_path}")

# Output each step's removal details
print(f"Number of rows removed due to missing values: {missing_values_count}, Rows: {missing_values_indices}")
print(f"Number of rows removed due to negative values: {negative_values_count}, Rows: {negative_values_indices}")
print(f"Number of rows removed due to outliers: {outliers_count}, Rows: {outliers_indices}")
print(f"Number of rows removed due to all-zero values: {all_zeros_count}, Rows: {all_zeros_indices}")
print(f"Number of rows removed due to duplicates: {duplicates_count}, Rows: {duplicates_indices}")


# 2. Cleaning_for_all_files_in_one_folder

In [ ]:
import os
import pandas as pd

# Define the input and output directories using the current directory
current_dir = os.getcwd()  
input_dir = current_dir 
output_cleaned_dir = os.path.join(current_dir, 'cleaned_files')  
output_removed_dir = os.path.join(current_dir, 'removed_files')  

# Create output directories if they don't exist
os.makedirs(output_cleaned_dir, exist_ok=True)
os.makedirs(output_removed_dir, exist_ok=True)

# Iterate over each file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # Load the dataset
        file_path = os.path.join(input_dir, filename)
        data = pd.read_csv(file_path)

        # Step 1: Handle Missing Values
        data_cleaned = data.copy()
        rows_to_remove = set(data_cleaned[data_cleaned.isna().any(axis=1)].index)
        missing_values_indices = list(rows_to_remove)
        missing_values_count = len(missing_values_indices)

        # Step 2: Remove Rows with Negative Values
        for col in data_cleaned.select_dtypes(include='number').columns:
            negative_indices = data_cleaned[data_cleaned[col] < 0].index
            rows_to_remove.update(negative_indices)
        negative_values_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices]
        negative_values_count = len(negative_values_indices)

        # Step 3: Handle Outliers by Removing Rows with Outliers (Based on Row Statistics)
        numeric_cols = data_cleaned.select_dtypes(include='number').columns
        for idx, row in data_cleaned[numeric_cols].iterrows():
            row_mean = row.mean()
            row_std_dev = row.std()
            outlier_threshold = 4 * row_std_dev

            # Check if any value in the row is an outlier
            if any((row > row_mean + outlier_threshold) | (row < row_mean - outlier_threshold)):
                rows_to_remove.add(idx)

        outliers_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices]
        outliers_count = len(outliers_indices)


        # Step 4: Remove Rows with 16 or more zeros (for numeric columns)
        zero_count_per_row = data_cleaned[numeric_cols].eq(0).sum(axis=1)  
        rows_to_remove.update(zero_count_per_row[zero_count_per_row >= 12].index)   
        all_zeros_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices and idx not in outliers_indices]
        all_zeros_count = len(all_zeros_indices)


        # Step 5: Remove Duplicate Records
        duplicates = data_cleaned[data_cleaned.duplicated(keep=False)].index
        rows_to_remove.update(duplicates)
        duplicates_indices = [idx for idx in rows_to_remove if idx not in missing_values_indices and idx not in negative_values_indices and idx not in outliers_indices and idx not in all_zeros_indices]
        duplicates_count = len(duplicates_indices)

        # Convert rows_to_remove to a list and sort it
        rows_to_remove = sorted(list(rows_to_remove))

        # Remove all identified rows from data_cleaned
        data_cleaned = data_cleaned.drop(index=rows_to_remove, errors='ignore')

        # Extract the removed rows from the original dataset
        removed_rows = data.loc[rows_to_remove]

        # Save the cleaned data and removed rows into new Excel files
        cleaned_file_path = os.path.join(output_cleaned_dir, f'cleaned_{filename.replace(".csv", ".xlsx")}')
        removed_rows_file_path = os.path.join(output_removed_dir, f'removed_{filename.replace(".csv", ".xlsx")}')

        # Save the cleaned data to an Excel file
        data_cleaned.to_excel(cleaned_file_path, index=False)

        # Save the removed rows to a separate Excel file
        removed_rows.to_excel(removed_rows_file_path, index=False)

        # Output each step's removal details for the current file
        print(f"Processed file: {filename}")
        print(f"Number of rows removed due to missing values: {missing_values_count}, Rows: {missing_values_indices}")
        print(f"Number of rows removed due to negative values: {negative_values_count}, Rows: {negative_values_indices}")
        print(f"Number of rows removed due to outliers: {outliers_count}, Rows: {outliers_indices}")
        print(f"Number of rows removed due to all-zero values: {all_zeros_count}, Rows: {all_zeros_indices}")
        print(f"Number of rows removed due to duplicates: {duplicates_count}, Rows: {duplicates_indices}")
        print(f"Cleaned data saved to: {cleaned_file_path}")
        print(f"Removed rows saved to: {removed_rows_file_path}")
        print("\n")


a little check for which rows were removed

In [ ]:
# Convert the set of removed row indices to a list
removed_rows_indices = list(rows_to_remove)

# Find the removed rows from the original dataset
removed_rows = data.loc[removed_rows_indices]

# Convert the removed rows to a numpy matrix
removed_rows_matrix = removed_rows.to_numpy()

# Display the shape of the numpy matrix to understand the size
removed_rows_matrix_shape = removed_rows_matrix.shape

# Output the result for user's reference
removed_rows_matrix_shape, removed_rows_matrix

# 3. Merge All buildings in one sheet and Uniform format

In [ ]:
import pandas as pd
import os

# Define the directory containing the input files (use current directory)
input_dir = os.getcwd()  # 当前文件夹路径
output_path = 'demo_form_recording.xlsx'

# Get a list of all Excel files in the directory
file_paths = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.xlsx')]

# Load all files into DataFrames and add a "Building" column to track the origin of each row
dfs = []
for file_path in file_paths:
    df = pd.read_excel(file_path)
    # Extract the building name from the file name
    building_name = os.path.basename(file_path).replace('.xlsx', '')
    # Add a column to identify the building each row came from
    df['Building'] = building_name
    dfs.append(df)

# Merge all dataframes into one for processing
combined_df = pd.concat(dfs, ignore_index=True)

# Extract the year and month from the 'Date' column and add 'year' and 'month' columns
combined_df['year'] = pd.to_datetime(combined_df['Date']).dt.year
combined_df['month'] = pd.to_datetime(combined_df['Date']).dt.month

# Extract unique years and months from the dataset
unique_years = sorted(combined_df['year'].unique())
unique_months = sorted(combined_df['month'].unique())

# Save the formatted results into a new Excel file with separate sheets for each year and month
with pd.ExcelWriter(output_path) as writer:
    for year in unique_years:
        for month in unique_months:
            # Filter the combined dataframe for the current year and month
            df_filtered = combined_df[(combined_df['year'] == year) & (combined_df['month'] == month)].drop(columns=['year', 'month'])

            # Define sheet name using year and month
            sheet_name = f'{year}_Month_{month}'
            
            # Write the formatted dataframe to a new sheet named after the year and month
            if not df_filtered.empty:  # Only write non-empty dataframes
                df_filtered.to_excel(writer, index=False, sheet_name=sheet_name)

print(f"Formatted data saved to: {output_path}")
